#Installations

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
print("Sucessful")

Sucessful


In [ ]:
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from functools import reduce
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import numpy as np

import re
import string

from pandas.io.json import json_normalize
from pyspark.sql.functions import explode, flatten
print("Sucessful")
import json


Sucessful




# Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Preprocessing Data with Pandas




##Reviews data preprocessing

### Read JSON data and load it in an object

In [ ]:
#Read JSON data and load it in an object
with open('/content/drive/My Drive/Data Science Stuff/Hotel Recommender/Data/reviews.json') as datafile:
    data_json = json.load(datafile)

###Normalize JSON data using Pandas

In [ ]:
#Normalize JSON data using Pandas
df_reviews_pd = pd.json_normalize(data_json)
df_reviews_pd.head()

,title,text,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,ratings.service,ratings.cleanliness,ratings.overall,ratings.value,ratings.location,ratings.sleep_quality,ratings.rooms,author.username,author.num_cities,author.num_helpful_votes,author.num_reviews,author.num_type_reviews,author.id,author.location,ratings.check_in_front_desk,ratings.business_service_(e_g_internet_access)
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,December 2012,93338,0,"December 17, 2012",147643103,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,Papa_Panda,22.0,12.0,29.0,24.0,8C0B42FF3C0FA366A21CFD785302A032,Gold Coast,NaN,NaN
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",December 2012,93338,0,"December 17, 2012",147639004,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,Maureen V,2.0,NaN,2.0,NaN,E3C85CA9DBBBC77E0DB534ABE93E4713,"Sydney, New South Wales, Australia",NaN,NaN
2,“Great Stay”,This is a great property in Midtown. We two di...,December 2012,1762573,0,"December 18, 2012",147697954,False,4.0,5.0,4.0,4.0,5.0,4.0,4.0,vuguru,12.0,17.0,14.0,14.0,FB1032DECE1162CB3556D05F278AAFFD,Houston,NaN,NaN
3,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,August 2012,1762573,0,"December 17, 2012",147625723,False,5.0,5.0,4.0,5.0,5.0,5.0,5.0,Hotel-Designer,5.0,26.0,5.0,5.0,EC3E275EE7590694889C8C7EE0D13961,"Laguna Beach, CA",NaN,NaN
4,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,December 2012,1762573,0,"December 17, 2012",147612823,False,4.0,5.0,4.0,3.0,5.0,5.0,5.0,JamesE339,34.0,65.0,104.0,49.0,BA524A238B1171206691A6CC3F28F266,"Saint Louis, Missouri",NaN,NaN


###View the data frame structure

In [ ]:
#Getting data structure/info
df_reviews_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878561 entries, 0 to 878560
Data columns (total 24 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   title                                           878561 non-null  object 
 1   text                                            878561 non-null  object 
 2   date_stayed                                     810973 non-null  object 
 3   offering_id                                     878561 non-null  int64  
 4   num_helpful_votes                               878561 non-null  int64  
 5   date                                            878561 non-null  object 
 6   id                                              878561 non-null  int64  
 7   via_mobile                                      878561 non-null  bool   
 8   ratings.service                                 760918 non-null  float64
 9   ratings.cleanliness       

###Drop redundent columns

In [ ]:
df_reviews_pd = df_reviews_pd.drop(columns=['date_stayed', 'num_helpful_votes', 'date', 'via_mobile', 'ratings.service', 'ratings.cleanliness', 'ratings.value', 'ratings.location', 'ratings.sleep_quality', 'ratings.rooms', 'author.num_cities', 'author.num_helpful_votes', 'author.num_reviews', 'author.num_type_reviews', 'author.location', 'ratings.check_in_front_desk', 'ratings.business_service_(e_g_internet_access)', 'author.username'])

###Rename columns

In [ ]:
df_reviews_pd = df_reviews_pd.rename(columns={"ratings.overall":"overall_rating", "author.id" : "author_id"})
df_reviews_pd.head()

,title,text,offering_id,id,overall_rating,author_id
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,93338,147643103,5.0,8C0B42FF3C0FA366A21CFD785302A032
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",93338,147639004,5.0,E3C85CA9DBBBC77E0DB534ABE93E4713
2,“Great Stay”,This is a great property in Midtown. We two di...,1762573,147697954,4.0,FB1032DECE1162CB3556D05F278AAFFD
3,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,1762573,147625723,4.0,EC3E275EE7590694889C8C7EE0D13961
4,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,1762573,147612823,4.0,BA524A238B1171206691A6CC3F28F266


###Removing Null values

####Check if we have any nan values

In [ ]:
df_reviews_pd.isna().sum()

title             0
text              0
offering_id       0
id                0
overall_rating    0
author_id         0
dtype: int64

In [ ]:
df_reviews_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878561 entries, 0 to 878560
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   title           878561 non-null  object 
 1   text            878561 non-null  object 
 2   offering_id     878561 non-null  int64  
 3   id              878561 non-null  int64  
 4   overall_rating  878561 non-null  float64
 5   author_id       878561 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 40.2+ MB


####Checking if our data has ' ' empty strings


In [ ]:
#returns the rows which has some empty string
df_reviews_pd[df_reviews_pd == ''].count()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


title                 0
text                  0
offering_id           0
id                    0
overall_rating        0
author_id         77066
dtype: int64

####Replace empty string ' ' with nan in whole data

In [ ]:
df_reviews_pd.replace('', np.nan, inplace=True)
#If you want to replace only in specific column
#df_reviews_pd['author_id'].replace('', np.nan, inplace=True)

####Now we have nan values as we replaced ' ' with nan

In [ ]:
df_reviews_pd.isna().sum()

title                 0
text                  0
offering_id           0
id                    0
overall_rating        0
author_id         77066
dtype: int64

In [ ]:
df_reviews_pd.count()

title             878561
text              878561
offering_id       878561
id                878561
overall_rating    878561
author_id         801495
dtype: int64

####Drop the rows which contain nan

In [ ]:
df_reviews_pd.dropna(inplace=True)

####Now we have clean data without any ' ' or nan

In [ ]:
df_reviews_pd.count()

title             801495
text              801495
offering_id       801495
id                801495
overall_rating    801495
author_id         801495
dtype: int64

###Viewing final DF Structure

In [ ]:
df_reviews_pd.info()
df_reviews_pd.count()
# df_reviews_pd.isnull().sum()
# df_reviews_pd.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801495 entries, 0 to 878560
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   title           801495 non-null  object 
 1   text            801495 non-null  object 
 2   offering_id     801495 non-null  int64  
 3   id              801495 non-null  int64  
 4   overall_rating  801495 non-null  float64
 5   author_id       801495 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 42.8+ MB


title             801495
text              801495
offering_id       801495
id                801495
overall_rating    801495
author_id         801495
dtype: int64

##Offering data preprocessing

### Read JSON data and load it in an object

In [ ]:
#Read JSON data and load it in an object
with open('/content/drive/My Drive/Data Science Stuff/Hotel Recommender/Data/offering.json') as datafileoffering:
    data_offering_json = json.load(datafileoffering)

###Normalize JSON data using Pandas

In [ ]:
df_offering_pd = pd.json_normalize(data_offering_json)
df_offering_pd.head()

,hotel_class,region_id,url,phone,details,type,id,name,address.region,address.street-address,address.postal-code,address.locality
0,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,,None,hotel,113317,Casablanca Hotel Times Square,NY,147 West 43rd Street,10036,New York City
1,5.0,32655,http://www.tripadvisor.com/Hotel_Review-g32655...,,None,hotel,76049,Four Seasons Hotel Los Angeles at Beverly Hills,CA,300 S Doheny Dr,90048,Los Angeles
2,3.5,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,,None,hotel,99352,Hilton Garden Inn Times Square,NY,790 Eighth Avenue,10019,New York City
3,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,,None,hotel,93589,The Michelangelo Hotel,NY,152 West 51st Street,10019,New York City
4,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,,None,hotel,217616,The Muse Hotel New York,NY,130 West 46th Street,10036,New York City


###View the data frame structure

In [ ]:
df_offering_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4333 entries, 0 to 4332
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hotel_class             3141 non-null   float64
 1   region_id               4333 non-null   int64  
 2   url                     4333 non-null   object 
 3   phone                   4333 non-null   object 
 4   details                 0 non-null      object 
 5   type                    4333 non-null   object 
 6   id                      4333 non-null   int64  
 7   name                    4333 non-null   object 
 8   address.region          4333 non-null   object 
 9   address.street-address  4331 non-null   object 
 10  address.postal-code     4303 non-null   object 
 11  address.locality        4333 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 406.3+ KB


###Drop redundent columns

In [ ]:
df_offering_pd = df_offering_pd.drop(columns=['hotel_class', 'region_id', 'url', 'phone', 'details', 'type', 'address.region', 'address.street-address', 'address.postal-code', 'address.locality'])
df_offering_pd.head()

,id,name
0,113317,Casablanca Hotel Times Square
1,76049,Four Seasons Hotel Los Angeles at Beverly Hills
2,99352,Hilton Garden Inn Times Square
3,93589,The Michelangelo Hotel
4,217616,The Muse Hotel New York


###Rename columns

In [ ]:
df_offering_pd = df_offering_pd.rename(columns={"name":"hotel_name", "id" : "hotel_id"})

In [ ]:
df_offering_pd.head()

,hotel_id,hotel_name
0,113317,Casablanca Hotel Times Square
1,76049,Four Seasons Hotel Los Angeles at Beverly Hills
2,99352,Hilton Garden Inn Times Square
3,93589,The Michelangelo Hotel
4,217616,The Muse Hotel New York


###Removing Null values


####Check if we have any nan values

In [ ]:
df_offering_pd.isna().sum()

hotel_id      0
hotel_name    0
dtype: int64

####Checking if our data has ' ' empty strings


In [ ]:
df_offering_pd[df_offering_pd == ''].count()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


hotel_id      0
hotel_name    0
dtype: int64


###Viewing final DF Structure

In [ ]:
# df_offering_pd.info()
# df_offering_pd.count()
# df_offering_pd.isnull().sum()
df_offering_pd.head()

,hotel_id,hotel_name
0,113317,Casablanca Hotel Times Square
1,76049,Four Seasons Hotel Los Angeles at Beverly Hills
2,99352,Hilton Garden Inn Times Square
3,93589,The Michelangelo Hotel
4,217616,The Muse Hotel New York


##Merging Data frames with Pandas

In [ ]:
df_merged = pd.merge(df_reviews_pd, df_offering_pd, how='left', left_on='offering_id', right_on='hotel_id')

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801495 entries, 0 to 801494
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   title           801495 non-null  object 
 1   text            801495 non-null  object 
 2   offering_id     801495 non-null  int64  
 3   id              801495 non-null  int64  
 4   overall_rating  801495 non-null  float64
 5   author_id       801495 non-null  object 
 6   hotel_id        801495 non-null  int64  
 7   hotel_name      801495 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 55.0+ MB


In [ ]:
df_merged.isna().sum()

title             0
text              0
offering_id       0
id                0
overall_rating    0
author_id         0
hotel_id          0
hotel_name        0
dtype: int64

In [ ]:
df_merged[df_merged == ''].count()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


title             0
text              0
offering_id       0
id                0
overall_rating    0
author_id         0
hotel_id          0
hotel_name        0
dtype: int64

###Dropping redundant columns

In [ ]:
df_merged.drop(columns=['offering_id'])

,title,text,id,overall_rating,author_id,hotel_id,hotel_name
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,147643103,5.0,8C0B42FF3C0FA366A21CFD785302A032,93338,Hotel Beacon
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",147639004,5.0,E3C85CA9DBBBC77E0DB534ABE93E4713,93338,Hotel Beacon
2,“Great Stay”,This is a great property in Midtown. We two di...,147697954,4.0,FB1032DECE1162CB3556D05F278AAFFD,1762573,Andaz 5th Avenue
3,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,147625723,4.0,EC3E275EE7590694889C8C7EE0D13961,1762573,Andaz 5th Avenue
4,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,147612823,4.0,BA524A238B1171206691A6CC3F28F266,1762573,Andaz 5th Avenue
...,...,...,...,...,...,...,...
801490,“Enthousiasmant”,"Très bien placé près de Dupont Circle, hotel t...",42161718,5.0,266F69FD8749B6C28198AE78EC1EDABB,84093,"Palomar Washington DC, a Kimpton Hotel"
801491,“Nos encanto...”,"Mi hija de 7 a~os , mi esposo y yo nos quedamo...",36621154,5.0,16A124ECD36140A18072FF2AF9F378F2,84093,"Palomar Washington DC, a Kimpton Hotel"
801492,“ottimo albergo”,IL Palomar è un ottimo quattro stelle in poszi...,19967730,4.0,5C02ABB2803B0D6E05803684426AB338,84093,"Palomar Washington DC, a Kimpton Hotel"
801493,“excellent”,"un excellent hotel 4 *, pour un prix raisonnab...",18003332,4.0,FF38DF65AA23823E729A05FA90C13FCE,84093,"Palomar Washington DC, a Kimpton Hotel"


###Rearranging the columns

In [ ]:
df_merged = df_merged.reindex(columns=['id', 'overall_rating', 'hotel_id', 'hotel_name','title', 'text', 'author_id',])

In [ ]:
df_merged.head()

,id,overall_rating,hotel_id,hotel_name,title,text,author_id
0,147643103,5.0,93338,Hotel Beacon,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,8C0B42FF3C0FA366A21CFD785302A032
1,147639004,5.0,93338,Hotel Beacon,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",E3C85CA9DBBBC77E0DB534ABE93E4713
2,147697954,4.0,1762573,Andaz 5th Avenue,“Great Stay”,This is a great property in Midtown. We two di...,FB1032DECE1162CB3556D05F278AAFFD
3,147625723,4.0,1762573,Andaz 5th Avenue,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,EC3E275EE7590694889C8C7EE0D13961
4,147612823,4.0,1762573,Andaz 5th Avenue,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,BA524A238B1171206691A6CC3F28F266


###Sorting Columns

In [ ]:
df_merged.sort_values(by=['id', 'hotel_id'], ascending=True, inplace=True)
df_merged.head(10)

,id,overall_rating,hotel_id,hotel_name,title,text,author_id
675509,683156,5.0,220216,Beacon Hill Hotel and Bistro,"“great hotel, great location”","The rooms here are not palatial, but they're v...",C0FE2026E16771D2C3322BB705FCA690
268288,692745,5.0,113317,Casablanca Hotel Times Square,“absolutely delightful”,I travelled from Spain to nyc in august 2002 a...,8C17DDA718DC7815A1B9ADDD9592EED3
661797,692780,2.0,100540,MarQueen Hotel,"“Marqueen, overrated?”",Sorry this is a bit long. In short: Location: ...,75EACF34E876B736647E8EAE0D9318B7
402255,692844,1.0,80133,Handlery Hotel San Diego,“a motel ' dreaming' to be considered as a hot...,Rooms have been recently renovated and i must ...,35FBC6EA978B4AE51536F1150F32C6AC
329375,692847,5.0,119443,The Westin Los Angeles Airport,“Wesin is more hotel for the money”,After a recent unpleasent stay at the Monte Ca...,5BE26753AC8336073525BC3819BA4D44
546777,692892,4.0,112303,Hotel Diva,"“Clean, Safe and Different”",The Diva hotel has a personality to it. It is ...,E01681A50DC19FFA18D84EC84718D867
8936,692980,4.0,223154,Hampton Inn Houston - Near The Galleria,"“Location, decent, and affordable”","OK, this is a Hampton Inn in the best sense. B...",38078EB1B0712938414B432CBA458F66
192670,693626,5.0,222959,Ritz-Carlton Battery Park,“One of the best!!”,What can you say about the Ritz Carlton? It's ...,8916B7F8AA251A7BDA27F5A1BC153D6F
727145,693665,5.0,223774,Madison Hotel,“Great New Accomodations”,finally...a new alternative to the crowded and...,F1EA76EBC9C5783D922FD61D32697FBD
418481,693714,1.0,80562,The Sheraton San Diego Hotel & Marina,“Don't bother.....”,We just returned from a weekend here in San Di...,889CA24F3FADE263B4FA96976AD441C1


#Data Testing

In [ ]:
# !pip install langdetect

In [ ]:
# from langdetect import detect

In [ ]:
# df_merged['Languagereveiw'] = df_merged['text'].apply(detect)
# df_merged[df_merged.text.apply(lambda x : detect(x))=='en']

In [ ]:
# df_merged[df_merged.id == '147801697'].head()

##Downloading the cleaned data

In [ ]:
# df_merged.to_json("/content/drive/My Drive/Data Science Stuff/Hotel Recommender/Data/merged.json", orient="records")

In [ ]:
df_merged.sort_values(by=['text'], ascending=False, inplace=True)
df_merged.head(10)

,id,overall_rating,hotel_id,hotel_name,title,text,author_id
393098,118469617,4.0,76741,Hilton San Diego Mission Valley,“ﾊﾞﾚｰﾋﾞｭｰが最高 ﾌﾟｰﾙも楽しめます”,ﾛｽよりもｻﾝﾃﾞｨｴｺﾞで長期滞在したい人にお勧め! ﾒｷｼｺとの国境の町 ｻﾝﾃﾞｨｴｺ...,BAD8F794E6055A84B9F935F29DD3D04A
307305,79205728,3.0,77714,DoubleTree by Hilton Hotel Los Angeles Downtown,“日本人にはよし”,ﾘﾄﾙﾄｰｷｮｰという立地上、ＬＡの観光スポットにいくにはとても不便。\nしかしスーパーや日...,2D9982954B3D19CE51A86D90AFCCF328
675128,135648138,5.0,224783,Grand Hyatt Seattle,“ＭＬＢをみるなら”,ｸﾗﾌﾞﾂｲﾝに3泊しました。ﾎﾃﾙはﾗｲﾄﾚｰﾙﾗｲﾝの終着駅(ﾊﾞｽ停も兼用）から歩いて...,7A5AFF5D8B936AFD97C40E047B64FA58
655423,123677205,3.0,100586,W Seattle,“期待はずれでした！”,Ｗホテルなので期待していましたが、ソファー・ベッド・壁紙等が老朽化していて\n残念でした。\...,98B84FBC9A653F1D2C66AF0177F1F432
208051,62733521,3.0,235238,W New York - Times Square,“ｗ タイムススクエアー”,Ｗは比較的新しくてきれいなのですがここはちょっと違いますね。古いホテルを買収してすべて改装は...,2A1718A0837433FC8B6EAFD0DDA87595
344846,114871537,4.0,78046,Sheraton Gateway Los Angeles,“空港の直ぐ傍で夜遅くとか朝早くのフライトを利用の際には大変便利です。”,Ｓｈｅｒａｔｏｎ hotelは時々利用しているが、空港からのシャトルサービス（無料ー荷物ある...,EE6E78042C9342A007554D930AE21B08
143602,45786094,4.0,611947,Hilton New York,“場所が最高”,ＮＹ観光は色々泊まりましたが、立地が大変良いと思います。さらにダイヤモンドメンバーは朝食無料...,02D60DACB188F821071A304A806D3056
141612,89623232,4.0,611947,Hilton New York,“ロケーション最高！そして屋台も！”,ＮＹ個人旅行ではどこに宿泊すべきか悩むところ・・・。なぜならご承知の通り、マンハッタン島のホ...,77FA484F573B9E3DCBDF7D21D85F54C3
466477,39063544,5.0,224221,Library Hotel,“とっても落ち着くホテルです”,ＮＹミッドタウンイーストにちょっとおしゃれなホテルがあります。\n市立図書館の近くにあるので...,30D1AF68F40F56D059D5A8CC469469BE
143581,49029469,4.0,611947,Hilton New York,“立地がいい”,ＮＹミッドタウンにあり、タイムズスクエアー、劇場、５番街、セントラルパーク、ＭＯＭA，などに...,8B08E7D0C6DB573A423BC8589476A57D


In [ ]:
df_merged['text'] = df_merged['text'].str.encode('ascii', 'ignore').str.decode('ascii')


In [ ]:
df_merged.head()

,id,overall_rating,hotel_id,hotel_name,title,text,author_id
393098,118469617,4.0,76741,Hilton San Diego Mission Valley,“ﾊﾞﾚｰﾋﾞｭｰが最高 ﾌﾟｰﾙも楽しめます”,! \nUSA(2008)\n !,BAD8F794E6055A84B9F935F29DD3D04A
307305,79205728,3.0,77714,DoubleTree by Hilton Hotel Los Angeles Downtown,“日本人にはよし”,\n\n\n\n,2D9982954B3D19CE51A86D90AFCCF328
675128,135648138,5.0,224783,Grand Hyatt Seattle,“ＭＬＢをみるなら”,3(23LRLLRL41530\nTVNHKWORLD CHANNELi-pod27F+ca...,7A5AFF5D8B936AFD97C40E047B64FA58
655423,123677205,3.0,100586,W Seattle,“期待はずれでした！”,\n\n,98B84FBC9A653F1D2C66AF0177F1F432
208051,62733521,3.0,235238,W New York - Times Square,“ｗ タイムススクエアー”,,2A1718A0837433FC8B6EAFD0DDA87595
